[![Video Placeholder](./placeholder.png)](https://youtu.be/6d0-8BbBvoM "Title Test")

For this - see article - https://devblogs.microsoft.com/powershell/powershell-foreach-object-parallel-feature/

# [wip]Title - Get more work done faster in PowerShell 7 with Parallelism

TODO - Intro Pragraph

TODO - Use case - what can they use this for

## What you need to follow along:

To follow along and try the domonstrations for yourself, you will need PowerShell version 7.0 or above.  If you're on Windows, PowerShell 7 runs side-by-side with Windows PowerShell 5.1 without impacting your previos work. Get the latest [here!](aka.ms/Get-PowerShell)

## Which foreach are we talking about?

PowerShell has two methods of iterating objects that use the word "foreach". One is the **ForEach-Object** cmdlet for interactive and scripting, the other is the **Foreach** keyword that has benefits in scripting, but it doesn;t handle pipeline input. Our focus is the
interactive and scripting cmdlet **ForEach-Object** that has the addition of parallelism in PowerShell 7.

For more information about these and their differences, see;

In [ ]:
Get-Help ForEach-Object -Full # This is the full help for ForEach-Object

In [ ]:
Get-Help About_Foreach # This is the full help about the keyword Foreach

## Quick review before we start...

TODO - text

```powershell
1..5 #
1..5 | ForEach-Object -Process {$_}
1..5 | ForEach-Object -Process {write-output "This is number $_"}
```

In [ ]:
1..5 | ForEach-Object -Process {write-output "This is number $_"}

TODO - Text - other ways you see this

```powershell
1..5 |  ForEach-Object {write-output "This is number $_"}
1..5 |  ForEach {write-output "This is number $_"}
1..5 | % {write-output "This is number $_"}
```

## Example (on Windows) Parallism to remote computers

TODO-text ( I want to kill processes that ive identified as possible problems....there might be 1 or a hundred on each computer)

```powershell
notepad;notepad;notepad # On Windows I'm launching several notepad processes - these will act as the offedning processes.
Get-Process -name note* # I would like to get a list of the processes notepad on the local machine as a test.
```

In [ ]:
notepad;notepad;notepad # On windows, this launches notepad 3 times
Get-Process -name note*

TODO - text - use Get-Member to determine the method or property you want to work with.

```powershell
Get-Process -name note* | Get-Member #this provides a list of methods and properties. I'm looking for something that would kill the process.
```

In [ ]:
Get-Process -name note* | Get-Member # Is there a method that might stop or kill a running process?

To complete stop or kill all process beginning with "note*" woudl like this:

```powershell
Get-Process -name note* | ForEach-Object {$_.kill}
```

## Using the power of Remoting for parallelism

TODO - text

```powershell
Invoke-Command -ComputerName comp1, comp2, comp3 {Get-Process -name note* | ForEach-Object {$_.kill}}
```

In [ ]:
Get-Process -name note* | ForEach-Object {$_.kill}

## Using the new parameter -Parallel for local performance improvements

TODO - text - not everything benefits form this.



## Moving large files

```powershell
# On Windows, without the parameter -Parallel
function Test-Copy {

    Measure-Command -Expression {
        1..5 | foreach-object -process {
            new-item -Path "$Home\dump\Dir_$_" -ItemType directory
            Copy-Item -Path "$Home\Demo\Source\*.*" -Destination "$Home\dump\Dir_$_\" -Force
        }
    } | Select-Object -Property Minutes, Seconds
}

```

```powershell
# On Windows, with the parameter -Parallel
function Test-Copy {

    Measure-Command -Expression {
        1..5 | foreach-object -parallel {
            new-item -Path "$Home\dump\Dir_$_" -ItemType directory
            Copy-Item -Path "$Home\Demo\Source\*.*" -Destination "$Home\dump\Dir_$_\" -Force
        }
    } | Select-Object -Property Minutes, Seconds
}
```

TODO - Text to demo ---

In [ ]:
Function Test-Copy {

    Measure-Command -Expression {
        1..5 | foreach-object -process {
            new-item -Path "$Home\dump\Dir_$_" -ItemType directory
            Copy-Item -Path "$Home\Demo\Source\*.*" -Destination "$Home\dump\Dir_$_\" -Force
        }
    } | Select-Object -Property Minutes, Seconds


In [ ]:
Function Test-Copy {

    Measure-Command -Expression {
        1..5 | foreach-object -parallel {
            new-item -Path "$Home\dump\Dir_$_" -ItemType directory
            Copy-Item -Path "$Home\Demo\Source\*.*" -Destination "$Home\dump\Dir_$_\" -Force
        }
    } | Select-Object -Property Minutes, Seconds


## Log analysis

TODO - text

```powershell
Function Test-Winlog {
    $logs = @(
        'System',
        'Application',
        'PowerShellCore/Operational',
        'Windows PowerShell',
        'Microsoft-Windows-PowerShell/Operational'
    )
    Measure-Command -Expression {
    $Logs | foreach-object -parallel {   # <----   Change from Process to Parallel - can set Throttle limit
        Get-Winevent -logname $_ |
            Where-Object -FilterScript {$_.LevelDisplayName -eq 'Error' -or $_.OpCodeDisplayName -eq 'Download'} |
            Sort-Object -Property TimeCreated -Descending |
            Select-Object -Property TimeCreated, ID, @{name='Note';Expression={$_.Message.Substring(0,60)}} |
            Add-Content -path c:\Demo\Log.Log -Force
    }
    } | Select-Object -Property Minutes, Seconds
}

```

In [ ]:
Function Test-Winlog {
    $logs = @(
        'System',
        'Application',
        'PowerShellCore/Operational',
        'Windows PowerShell',
        'Microsoft-Windows-PowerShell/Operational'
    )
    Measure-Command -Expression {
    $Logs | foreach-object -parallel {   # <----   Change from Process to Parallel - can set Throttle limit
        Get-Winevent -logname $_ |
            Where-Object -FilterScript {$_.LevelDisplayName -eq 'Error' -or $_.OpCodeDisplayName -eq 'Download'} |
            Sort-Object -Property TimeCreated -Descending |
            Select-Object -Property TimeCreated, ID, @{name='Note';Expression={$_.Message.Substring(0,60)}} |
            Add-Content -path c:\Demo\Log.Log -Force
    }
    } | Select-Object -Property Minutes, Seconds
}


## Note about throttling

TODO - text
TODO - example


## You try! 

1. Take a moment and try the following:


In [ ]:
# Clear this comment and try it here now!